In [1]:


from shapely.geometry import mapping, Polygon
import fiona

# You can see a schema as column names and types of a table which holds all the data (see bellow)

schema = {
    'geometry': 'Polygon',
    'properties': {'id': 'int'},
}

# Here are polygons which define our map

poly1 = Polygon([(0, 0), (0.1, 1), (1, 2), (0, 0)])
poly2 = Polygon([(0, 0), (2, 0), (1, 2), (0, 0)])

# Write all in a file (shapefile ou geojson)

#with fiona.open('/tmp/my_shp2.shp', 'w', 'ESRI Shapefile', schema) as c:
with fiona.open('/tmp/my_shp2.json', 'w', 'GeoJSON', schema) as c:
    c.write({
        'geometry': mapping(poly1),
        'properties': {'id': 1},
    })
    c.write({
        'geometry': mapping(poly2),
        'properties': {'id': 2},
    })



In [2]:
import json

with open('/tmp/my_shp2.json') as f:
    data = json.load(f)
    
data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {'id': 1},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 0.0], [0.1, 1.0], [1.0, 2.0], [0.0, 0.0]]]}},
  {'type': 'Feature',
   'properties': {'id': 2},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[0.0, 0.0], [2.0, 0.0], [1.0, 2.0], [0.0, 0.0]]]}}]}

In [3]:
import geopandas as gpd

shapes = gpd.read_file('/tmp/my_shp2.json')
shapes.plot(column='id')   # arg column is used to define colors, cmap maps column values to colors



ImportError: dlopen(/usr/local/lib/python3.7/site-packages/pyproj/_network.cpython-37m-darwin.so, 2): Symbol not found: _clock_gettime
  Referenced from: /usr/local/lib/python3.7/site-packages/pyproj/.dylibs/liblzma.5.dylib (which was built for Mac OS X 10.13)
  Expected in: /usr/lib/libSystem.B.dylib
 in /usr/local/lib/python3.7/site-packages/pyproj/.dylibs/liblzma.5.dylib